<a href="https://colab.research.google.com/github/enxo7899/INM701-Introduction-to-Artificial-Intelligence/blob/main/INM701.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import h5py
import numpy as np
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow.keras.utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn import metrics
import io
import os
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns


h5_file_path1 = './datasets/QuickDrawImages21.h5'
h5_file_path2 = './datasets/QuickDrawImages22.h5'
h5_file_path3 = './datasets/QuickDrawImages23.h5'
h5_file_path4 = './datasets/QuickDrawImages24.h5'
h5_file_path5 = './datasets/QuickDrawImages25.h5'

desired_images = {
    "202": "onion",
    "204": "owl",
    "208": "panda",
    "212": "parrot",
    "215": "pear",
    "218": "penguin",
    "220": "pickup_truck",
    "221": "picture_frame",
    "222": "pig",
    "223": "pillow",
    "224": "pineapple",
    "226": "pliers",
    "227": "police_car",
    "228": "pond",
    "229": "pool",
    "231": "postcard",
    "232": "potato",
    "235": "rabbit",
    "240": "rake",
    "242": "rhinoceros",
    "243": "river",
    "245": "rollerskates",
    "248": "saw"
    }

# categories = {
#     "onion": "healthyfood",
#     "owl": "bird",
#     "panda": "animal",
#     "parrot": "bird",
#     "pear": "healthyfood",
#     "penguin": "bird",
#     "pickup_truck": "vehicle",
#     "picture_frame": "rectangle",
#     "pig": "animal",
#     "pillow": "rectangle",
#     "pineapple": "healthyfood",
#     "pliers": "tool",
#     "police_car": "vehicle",
#     "pond": "water",
#     "pool": "water",
#     "postcard": "rectangle",
#     "potato": "heatlhyfood",
#     "rabbit": "animal",
#     "rake": "tool",
#     "rhinoceros": "animal",
#     "river": "water",
#     "rollerskates": "vehicle",
#     "saw": "tool"
# }

def add_desired_images(file_path, target_range_start, target_range_end, images_per_target, X, y, skip=0):
    with h5py.File(file_path, 'r') as h5_file:
        if 'images' in h5_file:
            images_dataset = h5_file['images']
            image_index = 0
            for target in range(target_range_start, target_range_end):
                if str(target) in desired_images:
                    for i in range(images_per_target):
                        # figure += 1
                        X.append(images_dataset[image_index])
                        y.append(target)
                        image_index += 1 + skip
                else:
                    image_index += images_per_target
        else:
            print("Image dataset not found!")
    return(X,y)

X = []
y = []

X,y = add_desired_images(h5_file_path1, 200, 210, 10000, X, y)
X,y = add_desired_images(h5_file_path2, 210, 220, 10000, X, y)
X,y = add_desired_images(h5_file_path3, 220, 230, 10000, X, y)
X,y = add_desired_images(h5_file_path4, 230, 240, 10000, X, y)
X,y = add_desired_images(h5_file_path5, 240, 250, 10000, X, y)

y = tensorflow.keras.utils.to_categorical(y)

# Split into training and test data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=42)
X_train = np.expand_dims(X_train, axis=3)
X_test = np.expand_dims(X_test, axis=3)

print(X_train[0][0])

num_classes = y.shape[1]
save_dir = './'
model_name = 'doodle.h5'

model = Sequential()
model.add(Conv2D(8, kernel_size=(3, 3), strides=1, padding='same', input_shape=(96, 96, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

history = model.fit(X_train,y_train,verbose=2,epochs=12)

pred_hot = model.predict(X_test)
pred = np.argmax(pred_hot,axis=1)
y_compare = np.argmax(y_test,axis=1) 
score = metrics.accuracy_score(y_compare, pred)

print("Accuracy score: {}".format(score))

print(pred_hot[:5])
print(pred)
